# Combining your data and ours
This notebook shows an example of how to:

- Extract a technical indicator from our Data Lake into a Pandas dataframe
- Import your own data in a queryable format
- Join the two datasets together in a SQL query


In [ ]:
# You must choose a particular feed and date to use for the queries in this notebook. 
# If you need a list of feeds, go to Help Menu -> "View Feed Data Permissions"

dt = '2025-02-05'
product = 'AAPL'
feed = 'xdp_nyse_integrated'

In [ ]:
! pip install duckdb

In [ ]:
from pathlib import Path

import duckdb
import maystreet_data as md
import numpy as np
import pandas as pd

First, we run an SQL query against the Data Lake to extract TWAP for crude oil futures by minute, and then store the result inside a Pandas dataframe.

You can also create an S3 bucket resource from the launcher, and store files there. Buckets can be shared between users, allowing to share datasets.

In [ ]:
query = f"""
-- TWAP

-- NOTE: "period_start" field is in UTC and period is currently 1 minute
WITH
    product_trades AS (
        SELECT
            price,
            exchangetimestamp,
            DATE_TRUNC(
                'minute',
                FROM_UNIXTIME(exchangetimestamp / 1e9)
            ) AS period_start
        FROM
            mt_trade
        WHERE
            f = '{feed}'
            AND dt = '{dt}'
            AND product = '{product}'
            -- if we wanted to filter by time of day
            -- AND FROM_UNIXTIME(exchangetimestamp / 1e9) BETWEEN TIMESTAMP '2022-09-18 14:30:00' AND TIMESTAMP '2022-09-18 21:00:00'
    ),
    period_twap_prep AS (
        SELECT
            exchangetimestamp,
            period_start,
            RANK() OVER (
                PARTITION BY period_start
                ORDER BY 
                    exchangetimestamp ASC
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) AS rnk,
            FIRST_VALUE(price) OVER (
                PARTITION BY period_start
                ORDER BY
                    exchangetimestamp ASC
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) AS first_price,
            LAST_VALUE(price) OVER (
                PARTITION BY period_start
                ORDER BY
                    exchangetimestamp ASC
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) AS last_price,
            MIN(price) OVER (
                PARTITION BY period_start
                ORDER BY exchangetimestamp ASC
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) AS min_price,
            MAX(price) OVER (
                PARTITION BY period_start
                ORDER BY exchangetimestamp ASC
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) AS max_price
        FROM
            product_trades
    ),
    max_ranks AS (
        SELECT
            period_start,
            MAX(rnk) AS max_rank
        FROM
            period_twap_prep
        GROUP BY
            period_start
    ),
    each_period as (
        SELECT
            prep.period_start,
            prep.first_price,
            prep.last_price,
            prep.min_price,
            prep.max_price
        FROM
            period_twap_prep prep
                JOIN
            max_ranks ranks ON prep.period_start = ranks.period_start AND prep.rnk = ranks.max_rank
    ),
    twap_components AS (
        SELECT
            period_start,
            (min_price + max_price + first_price + last_price) / 4 AS period_typical_price
        FROM
            each_period
    )
SELECT
    *,
    AVG(period_typical_price) OVER (ORDER BY period_start) AS twap
FROM
    twap_components
ORDER BY
    period_start
"""

twap_extract = pd.DataFrame(md.query(md.DataSource.DATA_LAKE, query))

Next, we import our data. In this case, we're randomly generating 100 bids and storing them inside a Pandas dataframe.

In practice, you'll want to use one of the following options to import real data:

- Store your data in your own S3 bucket, and allow the Workbench AWS role read access to it.
- Connect to your own storage from the workbench.
- If it's a small file, directly upload it by dragging the file into the Explorer in the Workbench interface.

In [ ]:
start = 1663538400000000000
end = start + 60 * 60 * 1000000000
timestamps = np.random.randint(start, end, 100)
bid = np.random.randint(8480, 8520, 100)
random_orders = pd.DataFrame(dict(timestamp=timestamps, bid=bid))

Finally, we use [DuckDB](https://duckdb.org/) to query the data together; note that DuckDB natively understands how to query data stored in a Pandas
dataframe in the same context without any additional configuration.

In this query, we're putting our bids together with the TWAP metrics in a single list, orderered by timestamp.

In [ ]:
query = """
SELECT
    random_orders.timestamp,
    random_orders.bid AS price,
    'bid' AS type
FROM
    random_orders
UNION ALL
SELECT
    twap_extract.period_start * 1000000 AS timestamp,
    twap_extract.twap AS price,
    'twap' AS type
FROM
    twap_extract
ORDER BY
    timestamp
LIMIT 100 -- only show the first 100 for this example
"""

with pd.option_context('display.max_rows', None):
    display(duckdb.query(query).df())
